# `aws rds`

`aws rds` (Relational Database Service) is a remotely hosted service for spinning up databases.

what follows will cover the most important elements of the `rds` service, and will losely follow [the `aws rds` documentation](http://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/Welcome.html), so look there for more details

## why `rds`?

it is absolutely possible to install a database on our remote `ec2` servers.

if we did that, we'd have total control over that database's configuration, security, resource usage, replication, archival... total control is a two-edged sword though: maybe we don't want the responsibility, or have the expertise?

we can let `aws` take on some of that burden

the tradeoff is the same as it has been for almost all of the `aws` services we've seen so far:

+ self-run: more granular control; but more admin overhead, complexity, and responsibility
+ managed: less admin overhead, complexity, and responsibility; but less granular control

additionally, it is almost guaranteed that the managed option will be easier and cheaper to operate at large scales (not that that matters to us in this moment!)

scaling to meet resource consumption or consumer demand, replication and backup, security, and other nice-to-have features are click-button options in `rds`

## what makes up an `rds` instance

an `rds` instance is a database, an availability zone, a security group, and parameterization

### database instances

each `rds` instance has a database instance -- actual database service software running on some `aws`-managed machine. 

as with `ec2` services before them, this means you must choose the hardware (memory and storage) and software (database service) that you want to use

#### hardware choices

the main dimensions here are

1. memory (will allow you to perform queries involving larger datasets)
2. cpu (will allow you to perform queries faster)
3. storage (will allow you to save more records)

at it's heart, you're doing the same thing you did when you set up your `ec2` server: identifying which of the above are important to you and selecting the instance type that performs best for your use case.

or just picking the free one. there's always that.

####  software choices

by "software" here we mean the installed sql database flavor. there are many flavors of relational databases, and `aws` supports some of the most popular:

1. `mysql`
2. `mariadb` (a special flavor / fork of `mysql`)
3. `postgres`
4. `oracle`
5. `mssql`
6. `amazon aurora` (an in-house modification of `mysql`)

### regions and availability zones

each instance you create will be located -- that is, physically -- in one or more (your choice) data centers in whatever region you select.

a *region* is a particular geographic area (*e.g.* US East 1, Northern Virginia).

an *availability zone* is an isolated datacenter within a given region. different availability zones are designed to be completely isolated, such that a problem or outage at one does not affect the others.

the default behavior (and the only free behavior) is to create *one* instance in *one* availability zone. in a sensitive and robust production setting, you would want to have multiple availability zones.

### security groups

just as with `ec2`, each database will need a security group to control access to and from this database. you can choose to open the database to the entire world, just your ip address, *etc*. it's the same song and dance.

### db parameter and option groups

there are *a lot* of configuration parameters and options associated with database management. `aws` separates them into two groups, and you created a normalized configuration object called a group for each type:

1. db paramater group
    1. these are parameters which determine how the database *itself* is configured
    2. car analogy: every car has an engine; the number of cylinders in the engine is a parameter of the engine
2. db option group
    1. these are parameters which determine how optional, extra features (database dependent) are configured, *if* they are activated or utilized
    2. car analogy: spoiler color; not every car has a spoiler, but if you want yours to be **AWESOME** and **FAST**, you should get a red one

## connecting two ways

once we've created an `rds` database, we will have *two* things we could connect to / interact with

1. the database itself
2. the `rds` service

generally speaking, when we discuss connections in what follows we're talking about connecting *to the database*. don't forget, though, that we can connect to and manipulate the `rds` service as well, via the `aws cli` or the `boto3` library.

## enough talk, let's make a database!

let's create some databases.

follow along while I create a `psql` database. start at the `rds` console: https://console.aws.amazon.com/rds/home

<br><div align="center">**walkthrough: create a `postgres` database**</div>

this is what happens in the above walkthrough:

1. navigate to [the `rds` console](https://console.aws.amazon.com/rds/home)
2. create a new `rds` instance
    1. click on the "get started now" button or the "Launch" icon on the Dashboard page, or the "Launch DB Instance" button on the "Instances" page
    2. engine choice
        1. click the "free usage tier" button
            1. note: this drops amazon aurora as an option, but *also* "step 2: Choose use case"
        2. unclick the "free usage tier" button
        3. let's go with `postgresql`
        4. click "next"
    3. use case
        1. this option is only available for non-free options
        2. the "Production" use case is not free because it comes with
            1. multi-az deployment (availability in multiple availability zones for increased redundancy and load balancing)
            2. provisioned iops storage
        3. we don't have production requirements -- this is about as dev/test-y as it gets.
        4. switch to "Dev/Test"
        5. click "next"
    4. db details
        1. click the "Free tier" checkbox
        2. pick the most recent db engine version
        3. observe: only one db option
        4. observe: multi-az deployment is not an option
        5. observe: only one storaget type option
        6. increase allocated storage to 10 GB
        7. fill in any info you want, but **don't forget the username and password**!
        8. click "Next"
    5. configure advanced settings
        1. network and security
            1. a VPC is an isolated network within the `aws` network, for full isolation within your apps and services
            2. select "Yes" for "Public accessibility"
            3. "No preference" for AZ
            4. leave "Create new" selected for the Security group
        2. database
            1. one `postgresql` *server* may have several *databases* (isolated collections of tables, table schemas, users, permissions, indices, etc)
            2. leave the other defaults
        3. encryption
            1. same as with `s3`, encrypts on local machine
            2. apps which access this db see *decrypted* values only
        4. backup
            1. set the retention period to be 1 day
            2. no preference for backup window
        5. monitoring
            1. only useful if you want to see system resources (e.g. which query is causing me to burn through iops)
        6. maintenance
            1. takes care of patching database software
        7. click "Launch DB Instance"
    6. click "View DB Instance details"

### connecting to our db: `pgadmin` gui

while `aws` is spinning up that postgres database (takes a few minutes), go install the `pgadmin` program:

https://www.pgadmin.org/download/

install the most recent non-release candidate of `pgadmin 4` (or the RC, if you're feeling saucy)

<br><div align="center">**mini exercise: download and install `pgadmin`**</div>

`pgadmin` is not the *only* gui available for interacting with `postgres` databases, but it is the one developed by and supported by the `postgres` project -- hence my choice here.

there are a lot of other options, some of which have a prettier interface and more features. If you're looking for bells and whistles, give [one of these](https://wiki.postgresql.org/wiki/Community_Guide_to_PostgreSQL_GUI_Tools) a shot (and [here, a list with screencaps](https://www.quora.com/What-is-a-good-GUI-tool-for-PostgreSQL-that-has-a-comfy-tabbed-view-for-results-and-queries-easy-sorting-and-grouping-of-results-IntelliSense-a-good-%E2%80%9Cquery-builder%E2%80%9D-to-help-with-syntax-support-for-versioning-of-SQL-code-and-a-good-debug-mode))

alright! status check: we should have the following:

1. `pgadmin` installed
2. a `postgresql` database in an "available" state
3. a security group accepting your *current* ip address

let's connect to our db with `pgadmin`

<div align="center">**mini exercise: connect to your `postgresql` instance using `pgadmin`**</div>

1. open your `psql` server to your current `ip`
    1. find your `psql` server's security group
    2. add an `inbound` rule of `postgresql` type from this or all `ip` addresses
2. open your `psql` server using `pgadmin`
    1. "Add New Server" button
    2. "Host" --> "Endpoint" and "Port" --> "Port", "Username" and "Password" from memory
3. create a table
    1. navigate to your database > Schema > Public > Tables
    2. right click and "create new table"

you probably know what's next by now: we just connected to a thing we built using a nice gui with helpful pictures and drop down menus and click-button options, so naturally it is time to eschew all of those finer things and access the same database from *the command line only*!

### connecting to our db: `psql cli`

really, though, we will often want to execute scripts directly from the command line, or do simple querying while developing, our use connecting libraries from within our `python` or `R` scripts, so it's good to build some familiarity with accessing databases from the `cl`

<div align="center">**mini exercise: install `psql` command line client**</div>

1. open your `ec2` server
2. install: `sudo apt install postgresql-client`
3. verify: `which psql`
4. rtfm: `man psql`
    1. figure out how to specify `host`, `port`, `username`, `dbname`, and `password`

the `psql` command is yet another programming language shell. just like the `python` or `R` command, this will open a `shell` (an environment where certain typed commands will do certain things) and allow us to write `postgres sql` statements to query a database.

<div align="center">**mini exercise: connecting to our `postgres` server using `psql` command line client**</div>

connect to your database by passing the `host`, `port`, `username`, and `dbname`, and then typing the `password`:

```bash
psql --host HOSTNAME \
     --port PORT_NUMBER \
     --username USER_NAME \
     --dbname DATABASE_NAME
```

what happened? why do you think that happened?

<div align="center">**mini exercise: add your `ec2` servers to your security group and *then* connecting **</div>

1. go to your `rds` instance's page
2. select the `security group`
3. edit the `inbound` rules
    1. add TCP access to port 5419 for your `ec2` server's IP address or security group name
4. try to connect from your `ec2` server again

```bash
psql --host HOSTNAME --port PORT_NUMBER --username USER_NAME --dbname DATABASE_NAME
```

notice a difference?

note: if you don't pass `dbname` explicitly, `psql` will try to connect to a database with name `USER_NAME` by default. this will usually not exist, but it's not uncommon to forget this and receive a

```
psql: FATAL:  database "USER_NAME" does not exist
```

error

#### a review of `psql` commands

when in the `psql` shell, you effectively have access to two languages:

1. traditional `sql`: `SELECT ... FROM ... WHERE` and the `LIKE`
2. `psql` commands
    1. these all start with the `\` character (e.g. `\h`, or `\password`)

to get an idea of the options available, try the following:

1. `\h`: a general `sql` help function
    1. on its own, it lists all the `sql` commands available
2. `\h SQL_COMMAND`: a concise help menu for a given `sql` command
    1. example: `\h select`
3. `\?`: lists all the `psql` commands available with a short description
    1. example: `\d`
    2. example: `\l`

<div align="center">**mini exercise: create a new table in our database**</div>

1. first, make sure you're connected to the database you think you are
    1. check the prompt: `databasename=>`, or the results of command `\c`
2. list the existing tables (relations): `\d`
3. execte the below `sql` statement
4. run `\d` and `select` lines from your table (there will be none unless you `insert` your own)

```sql
create table YOUR_NAME_HERE (
    id serial not null
    , name varchar
);
```

+ Purchase methods:
    + Up-front reserved
    + On-demand hourly
    + spot

+ exercises
    + dynamodb
        + insert event, lambda function to trigger email sent or some file being written to s3 or something silly
        + aws dynamodb and boto3 dynamodb example cases